In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="dark")

### Environment

**STATE**

**ACTIONS**

**REWARD**


### Build Agent

In [ ]:
def build_Deep_Q_Agent():
    model = Sequential()
    model.add(Dense(64, input_dim=state_size, activation='relu'))
    model.add(Dense(32, input_dim=state_size, activation='relu'))
    model.add(Dense(action_size, activation='linear')) # LEFT OR RIGHT
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=['acc'])
    return model

### Perform action from given state

In [ ]:
def perform_action(model, state, epsilon):
    action = -1
    
    if np.random.rand() <= epsilon:
        action = random.randrange(3)
    else:
        action = agent.predict(state.reshape(-1, 2))
        action = np.argmax(action)
        
    return action

### Remember transition

In [ ]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

### Train agent to maximize future reward

In [ ]:
def replay(batch_size, epsilon, done):
    
    # Get a minibatch of previous training memories
    minibatch = random.sample(memory, batch_size)
    
    #print("Replay: Train model on sample of previous memories")

    # For each memory
    for state, action, reward, next_state, done in minibatch:
        
        if not done:
            # Predict future disconted reward if not a terminating step
            target = (reward + gamma * np.amax(agent.predict(next_state)[0]))
        else:
            # Terminating step, no prediction needed
            target = reward
        
        # Map an approximation between current state to the future discounted reward
        target_f = agent.predict(state)
        target_f[0][action] = target
        
        # Train model to find map function from state to target
        # This will maximize the future reward based on current state
        agent.fit(state, target_f, epochs=1, verbose=0)
    
    # Decrease exploration successively
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
    
    return epsilon

### Load and Save

In [ ]:
def load(model, name):
    return model.load_weights(name)

def save(model, name):
    model.save_weights(name)

### Params

In [ ]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n # left, right
memory = deque(maxlen=1000)
gamma = 0.95    # discount rate
epsilon = 1  # exploration rate
epsilon_min = 0.01 # min exploration rate
epsilon_decay = 0.995 # decrease exploration successively
learning_rate = 0.005
episodes = 1000
batch_size = 10
max_score = -1.2
max_score_ep = 0

## Run

#### Pre-training

In [ ]:
gaming_score_threshold = -195
nbr_of_games = 10000
nbr_of_steps_per_game = 200
        
env.reset()

def train_agent():
    
    game_memory = []
    scores = []
    
    for game in range(nbr_of_games):
        
        env.reset()
        game_score = 0
        memory_current_game = []
        state = []
        next_state = [] 
        
        for step in range(nbr_of_steps_per_game):
            
            action = random.randrange(0, 3)
            next_state, reward, done, info = env.step(action)
            
            if len(state) > 0:
                memory_current_game.append((state, action))

            if next_state[0] > -0.25:
                reward = 0.5
                
            next_state = np.reshape(next_state, [1, state_size])    
            state = next_state
            game_score += reward
            
            if done: 
                #print(game_score)
                break
 
        if game_score >= gaming_score_threshold:
        
            scores.append(game_score)
            for action in memory_current_game:
                if action[1] == 0:
                    target = [1, 0, 0]
                elif action[1] == 1:
                    target = [0, 1, 0]
                elif action[1] == 2:
                    target = [0, 0, 1]
                game_memory.append((action[0], target))
                print("Added")
        
    return scores, game_memory

In [ ]:
scores, game_memory = train_agent()

In [ ]:
len(game_memory)

In [ ]:
X = np.array([m[0] for m in game_memory]).reshape(-1, 2)
y = np.array([m[1] for m in game_memory]).reshape(-1, 3)
agent = build_Deep_Q_Agent()
agent.fit(X, y, epochs=5)
agent.summary()

#### Run

In [ ]:
done = False
results = []
epsilon = 0.25

for episode in range(episodes):

    state = env.reset()
    state = np.reshape(state, [1, state_size])
    max_score = 0
    score = 0

    for trial in range(200):
        
        env.render()
        
        action = perform_action(agent, state, epsilon)
        next_state, reward, done, info = env.step(action)
        state = next_state
        
        if done and trial < 199: 
            print("Victory!")
            break
        if done:
            print("Loss!")
            break
    
    env.reset()
    results.append(score)


#### Non Pre-training

In [ ]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n # left, right
memory = deque(maxlen=20000)
gamma = 0.95    # discount rate
epsilon = 1  # exploration rate
epsilon_min = 0.01 # min exploration rate
epsilon_decay = 0.995 # decrease exploration successively
learning_rate = 0.005
episodes = 1000
batch_size = 100
max_score = -1.2
max_score_ep = 0

In [ ]:
#agent = build_Deep_Q_Agent()
#agent = load(agent, 'weights/cartpole-dqn.h5')
epsilon = 0.01
done = False
results = []

for episode in range(episodes):

    state = env.reset()
    state = np.reshape(state, [1, state_size])
    max_score = 0
    score = 0
    print("Next episode!")

    for trial in range(200):
        
        env.render()
        
        action = perform_action(agent, state, epsilon)
        next_state, reward, done, info = env.step(action)

        state_pos = next_state[0]
        
        if state_pos > -0.25:
            reward = 0.5     
            
        next_state = np.reshape(next_state, [1, state_size])
        if state_pos > -0.4:
            remember(state, action, reward, next_state, done) 
        #state = next_state
        
        if done:
            
            print("\nEpisode: {}/{}, Score: {}, Epsilon: {:.3}, Top score: {}".format(episode, episodes, score, float(epsilon), max_score))
            print("Trial: ", trial)
            #results.append(max_score)
            
            #if episode % 1 == 0:
            #    print("Average score: ", sum(results[-25:]) / 25.0) 
            
            break
            
        # ---------------------------------------------------------
        # Sample random minibatches from memory and train model to 
        # maximize future reward from current state
        # ---------------------------------------------------------
    
        if len(memory) > 5000:
            epsilon = replay(batch_size, epsilon, done)
            
    print("Size: ", len(memory))

    #if episode % 50 == 0:
    #    print("Save weights!")
    #    save(agent, "weights/mountain-car-dqn-" + str(episode) + ".h5")